# Experiments with non-EuroSAT satellite images

In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2
import random

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Load images

In [2]:
def load_images(base_path, classes, num_samples, train=True):
    images = []
    labels = []

    subset = 'train/train' if train else 'test/test'
    base_path = os.path.join(base_path, subset)

    for cls in classes:
        cls_path = os.path.join(base_path, cls)
        img_files = os.listdir(cls_path)
        
        # Pick random images from the class
        selected_files = random.sample(img_files, num_samples)
        
        for img_file in selected_files:
            img_path = os.path.join(cls_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(cls)

    return images, labels

base_path = 'dataset/NWPU-RESISC45'
classes = ['forest', 'river', 'meadow', 'dense_residential', 'industrial_area', 'freeway', 'mountain', 'island', 'circular_farmland', 'rectangular_farmland']
num_samples = 500

images, labels = load_images(base_path, classes, num_samples)

In [3]:
mapping = {
    'forest': 'Forest',
    'river': 'River',
    'dense_residential': 'Residential',
    'meadow': 'Pasture',
    'industrial_area': 'Industrial',
    'freeway': 'Highway',
    'mountain': 'HerbaceousVegitation', 
    'island': 'SeaLake', 
    'circular_farmland': 'PermanentCrop', 
    'rectangular_farmland': 'AnnualCrop',
}
mapped_labels = [mapping[label] for label in labels]


In [4]:
# Preprocess the images and labels
images = np.array(images) / 255.0
labels_encoded = pd.get_dummies(mapped_labels).values
y_true = np.argmax(labels_encoded, axis=1)

## ConvNeXt

In [5]:
## load model
from tensorflow.keras.models import load_model
model_1 = load_model("model_1_nFull_epoch5_batch32")

In [6]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_1.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 2s 2s/step


In [7]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 54.58%


In [8]:
class_names = list(pd.get_dummies(mapped_labels).columns)
model_class_indices = list(range(10))  # List of indices for the 10 classes in the model

In [9]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names, labels=model_class_indices))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.38      0.27      0.32       500
              Forest       0.61      0.78      0.68       500
HerbaceousVegitation       0.76      0.87      0.81       500
             Highway       0.61      0.65      0.63       500
          Industrial       0.55      0.77      0.64       500
             Pasture       0.19      0.10      0.13       500
       PermanentCrop       0.35      0.43      0.39       500
         Residential       0.60      0.18      0.28       500
               River       0.66      0.52      0.58       500
             SeaLake       0.57      0.89      0.69       500

            accuracy                           0.55      5000
           macro avg       0.53      0.55      0.52      5000
        weighted avg       0.53      0.55      0.52      5000



In [10]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[135   1   5  22   4 110 199  12   6   6]
 [  0 388  36  20   2   3   1   1   9  40]
 [  0  45 434   1   0  10   2   0   0   8]
 [  3   2   1 325  49   3  21   0  18  78]
 [  0   0   0  41 384   0  28  39   8   0]
 [  2 188   7  47   0  52   0   4  17 183]
 [212   0   1  11   0  58 215   0   0   3]
 [  0   0   0  12 258   6  87  92  43   2]
 [  0  15  66  51   0  29  55   6 260  18]
 [  0   0  18   0   0   6   2   0  30 444]]


In [11]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [12]:
# ## My matplotlip is broken so im using something else. 
# ## Feel free to use this if you want. 

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 7))
# sns.set(font_scale=1.4)  # Adjust font size
# sns.heatmap(cm_df, annot=True, annot_kws={"size": 12}, cmap="YlGnBu", fmt="g", linewidths=.5, cbar=False)
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.title('Confusion Matrix Heatmap')

# plt.show()


In [13]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ViT

In [14]:
def load_images(base_path, classes, num_samples, train=True):
    images = []
    labels = []

    subset = 'train/train' if train else 'test/test'
    base_path = os.path.join(base_path, subset)

    for cls in classes:
        cls_path = os.path.join(base_path, cls)
        img_files = os.listdir(cls_path)
        
        # Pick random images from the class
        selected_files = random.sample(img_files, num_samples)
        
        for img_file in selected_files:
            img_path = os.path.join(cls_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(cls)

    return images, labels

base_path = 'dataset/NWPU-RESISC45'
classes = ['forest', 'river', 'meadow', 'dense_residential', 'industrial_area', 'freeway', 'mountain', 'island', 'circular_farmland', 'rectangular_farmland']
num_samples = 500

images_vit, labels = load_images(base_path, classes, num_samples)

In [15]:
# Preprocess the images and labels
images_vit = np.array(images_vit) / 255.0
labels_encoded = pd.get_dummies(mapped_labels).values
y_true = np.argmax(labels_encoded, axis=1)

In [16]:
import tensorflow_addons as tfa
from tensorflow.keras.models import load_model

custom_objects = {
    "AdamW": tfa.optimizers.AdamW
}

model_2 = load_model("model_vit_classifier", custom_objects=custom_objects)

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning:



TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 




In [17]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images_vit), batch_size):
    batch_pred = model_2.predict(images_vit[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 1s 795ms/step


In [18]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 26.70%


In [19]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names, labels=model_class_indices))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.16      0.03      0.05       500
              Forest       0.34      0.61      0.44       500
HerbaceousVegitation       0.13      0.51      0.21       500
             Highway       0.14      0.09      0.11       500
          Industrial       0.54      0.45      0.49       500
             Pasture       0.03      0.00      0.01       500
       PermanentCrop       0.24      0.41      0.30       500
         Residential       0.73      0.21      0.32       500
               River       0.32      0.06      0.10       500
             SeaLake       0.91      0.30      0.45       500

            accuracy                           0.27      5000
           macro avg       0.35      0.27      0.25      5000
        weighted avg       0.35      0.27      0.25      5000



In [20]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 16  49 230   2   2  14 183   2   0   2]
 [  8 304 158   2   0   0  28   0   0   0]
 [  6  39 254   3   1   2 193   2   0   0]
 [  2   2 418  43  11   0  18   5   1   0]
 [  0   0 122  98 226   0  26  26   2   0]
 [ 12 436  43   0   0   2   7   0   0   0]
 [ 22  10 213   7   1  15 205   0  18   9]
 [  0   0 222  71  33   0  70 104   0   0]
 [ 18  46 239  12   4  20 124   3  30   4]
 [ 16   9  39  73 140  14  15   1  42 151]]


In [21]:
cm = confusion_matrix(y_true, y_pred_classes)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

In [22]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ResNet50

In [23]:
## load model
from tensorflow.keras.models import load_model
model_resnet = load_model("model_resnet50_nFull_epoch20_batch32_lr.00001")

In [24]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_resnet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)
 

1/1 [==============================] - 1s 794ms/step


In [25]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 12.20%


In [26]:
class_names = list(pd.get_dummies(classes).columns)

In [27]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

   circular_farmland       0.10      0.10      0.10       500
   dense_residential       0.00      0.00      0.00       500
              forest       0.00      0.00      0.00       500
             freeway       0.10      0.09      0.10       500
     industrial_area       0.24      0.47      0.31       500
              island       0.76      0.04      0.07       500
              meadow       0.10      0.00      0.01       500
            mountain       0.08      0.38      0.13       500
rectangular_farmland       0.12      0.00      0.00       500
               river       0.16      0.13      0.14       500

            accuracy                           0.12      5000
           macro avg       0.17      0.12      0.09      5000
        weighted avg       0.17      0.12      0.09      5000



In [28]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 49   4   0  49  34   2   2 251   1 108]
 [ 82   0   5  15  43   1   5 337   0  12]
 [ 34   0   0  33  73   0   7 334   0  19]
 [ 31   8   1  45  47   0   1 255   1 111]
 [  0   3   0   4 237   0   0 227   0  29]
 [186   4   0  23   0  19   1 257   0  10]
 [ 44   3   0  84 148   0   2 186   5  28]
 [  0  32   0   0 257   0   0 191   0  20]
 [ 20   1   0  72  90   1   2 299   1  14]
 [ 44  18   0 116  78   2   0 176   0  66]]


In [29]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## MobileNet

In [30]:
## load model
from tensorflow.keras.models import load_model
model_mobilenet = load_model("model_MobileNet_nFull_epoch10_batch32")

In [31]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_mobilenet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 0s 388ms/step


In [32]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 38.24%


In [33]:
class_names = list(pd.get_dummies(classes).columns)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

   circular_farmland       0.23      0.02      0.04       500
   dense_residential       0.18      0.18      0.18       500
              forest       0.54      0.96      0.69       500
             freeway       0.26      0.41      0.32       500
     industrial_area       0.50      0.88      0.64       500
              island       0.08      0.00      0.00       500
              meadow       0.29      0.46      0.36       500
            mountain       0.27      0.35      0.31       500
rectangular_farmland       0.33      0.14      0.20       500
               river       0.94      0.40      0.57       500

            accuracy                           0.38      5000
           macro avg       0.36      0.38      0.33      5000
        weighted avg       0.36      0.38      0.33      5000



In [34]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[ 12   4   1  22  16   4 435   6   0   0]
 [  0  91 178   3   6   0  15 204   1   2]
 [  0   5 480   7   0   0   5   3   0   0]
 [ 18  11   8 205  73   0  28  81  67   9]
 [  0   0   0   4 440   0   1  55   0   0]
 [ 11 326  29  58   7   1  44  21   3   0]
 [ 12   1   7 214  19   1 232  14   0   0]
 [  0   0   1  12 301   0   9 177   0   0]
 [  0   7  90 229  14   1  26  60  72   1]
 [  0  48  89  43   5   6   0  34  73 202]]


In [35]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()
